Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [2]:
#!pip install "scikit-surprise==1.1.3"

In [3]:
#!pip install "xlrd==2.0.1"

### Import

In [13]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold

import seaborn as sns
from surprise import SVD, KNNWithMeans
from surprise import dump
np.random.seed(42)

### Базовые функции для скоринга и получения рекомендаций

### Загрузка и обработка данных

In [5]:
df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [6]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [7]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [8]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [9]:
# обучим с лучшими параметрами
algo_svd = SVD(random_state=0, n_epochs=100, n_factors= 512)
algo_svd.fit(trainset_data)

# получим предикт и посмотрим метрику
predictions_svd = algo_svd.test(trainset_data.build_testset())
accuracy.rmse(predictions_svd)

RMSE: 0.2524


0.25238153948056485

In [10]:
# обучим с лучшими параметрами
#algo_knn = KNNWithMeans(sim_options={'name': 'cosine', 'min_support': 3, 'user_based': False})
#algo_knn.fit(trainset_data)

# получим предикт и посмотрим метрику
#predictions_knn = algo_knn.test(trainset_data.build_testset())
#accuracy.rmse(predictions_knn)

In [16]:
file_name = r'surprise\baseline_svdpp_tuning_all_data'
#dump.dump(file_name, algo=algo)
_, algo_svdpp = dump.load(file_name)

In [18]:

# получим предикт и посмотрим метрику
predictions_svdpp = algo_svdpp.test(trainset_data.build_testset())
accuracy.rmse(predictions_svdpp)

RMSE: 0.4238


0.42381013577501886

In [19]:
df['Rating_svd'] = [x.est for x in predictions_svd]
df['Rating_svdpp'] = [x.est for x in predictions_svdpp]

df['err_svd'] = np.abs(df['Rating_svd'] - df['Rating'])
df['err_svdpp'] = np.abs(df['Rating_svdpp'] - df['Rating'])
df

,UID,JID,Rating,Rating_svd,Rating_svdpp,err_svd,err_svdpp
0,1,1,-7.82,-7.586908,-7.593449,0.233092,0.226551
1,1,2,8.79,8.382193,8.421380,0.407807,0.368620
2,1,3,-9.66,-9.501438,-9.464442,0.158562,0.195558
3,1,4,-8.16,-8.181290,-8.163395,0.021290,0.003395
4,1,5,-7.52,-7.368126,-7.377866,0.151874,0.142134
...,...,...,...,...,...,...,...
1448359,24983,67,6.21,6.119293,6.169552,0.090707,0.040448
1448360,24983,68,7.48,7.374133,7.430004,0.105867,0.049996
1448361,24983,69,5.15,5.124204,5.135486,0.025796,0.014514
1448362,24983,71,6.26,6.168962,6.237052,0.091038,0.022948


In [21]:
df_gr = df.groupby('UID').agg({'err_svd':'mean', 'err_svdpp':'mean'})
df_gr

,err_svd,err_svdpp
UID,,
1,0.160365,0.150069
2,0.176008,0.155665
3,0.081557,0.075191
4,0.094060,0.102181
5,0.142172,0.135265
...,...,...
24979,0.126711,0.072547
24980,0.116749,0.042033
24981,0.065816,0.036046


In [24]:
users = set()

for i, row in enumerate(df_gr.values):
    err_svd = row[0]
    err_svdpp = row[1]
    
    users.add(i)
    if err_svdpp < err_svd:
        print(i)

0
1
2
4
5
6
7
8
11
14
16
19
53
58
76
82
123
4111
4149
5062
5438
5677
6009
6023
6561
6615
6625
6664
6702
6734
6737
6742
6915
6927
6946
7093
7222
7255
7340
7377
7404
7644
7659
7694
7731
7737
7784
7936
8098
8102
8106
8141
8154
8195
8253
8328
8335
8367
8397
8421
8429
8451
8483
8497
8568
8621
8622
8676
8684
8702
8725
8735
8759
8763
8843
8921
8959
8967
9087
9098
9180
9185
9193
9311
9359
9366
9467
9506
9510
9550
9604
9639
9667
9712
9714
9740
9766
9768
9819
9828
9870
9903
9917
10026
10064
10067
10078
10086
10112
10118
10142
10186
10294
10310
10434
10447
10658
10694
11389
11583
11588
11609
11622
11632
11654
11655
11675
11688
11739
11766
11771
11773
11798
11804
11825
11856
11886
11916
11920
11945
11991
11993
12005
12049
12075
12099
12104
12190
12199
12234
12242
12244
12346
12519
12699
12775
12861
13184
13343
13354
13430
13440
13651
13728
13783
13832
14125
14176
14181
14188
14232
14248
14362
14429
14451
14465
14641
14663
14738
14795
14810
14821
14825
14958
14964
15022
15067
15107
15110
15168
1519

In [26]:
len(users)

24983

In [24]:
test = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [25]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [26]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,3.042851
1,-7.039993
2,-0.331728
3,6.796138
4,5.779301


In [27]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline_svd_tuning_best_4.02408.csv')